In [8]:
# Import libraries
import numpy as np
import pandas as pd

In [33]:
# read in the json files
offers = pd.read_json('raw_data/portfolio.json', orient='records', lines=True)
customers = pd.read_json('raw_data/profile.json', orient='records', lines=True)
events = pd.read_json('raw_data/transcript.json', orient='records', lines=True)

# change column names and print df shape
offers.columns = ['reward','channels','min_spend','expire_days','offer_type','id']
customers.columns = ['gender','age','id','became_member_on','income']
events.columns = ['customer_id','event','value','hour']
print(f'offers has {len(offers)} rows')
print(f'customers has {len(customers)} rows')
print(f'events has {len(events)} rows')

offers has 10 rows
customers has 17000 rows
events has 306534 rows


In [34]:
customer_funnel = events.groupby(['customer_id','event']).event.count().to_frame().rename({'event':'count'},axis=1).reset_index()
customer_funnel = pd.pivot(customer_funnel, index='customer_id', columns='event', values='count')
deleted_customers = customer_funnel[customer_funnel['offer received'].isnull()].index

events = events[~events.customer_id.isin(deleted_customers)]
customers = customers[~customers.id.isin(deleted_customers)]
events.reset_index(inplace=True, drop=True)
customers.reset_index(inplace=True, drop=True)

print(f'Removed {len(deleted_customers)} customers from all datasets who did not get any offers')
print(f'offers has {len(offers)} rows')
print(f'customers has {len(customers)} rows')
print(f'events has {len(events)} rows')

Removed 6 customers from all datasets who did not get any offers
offers has 10 rows
customers has 16994 rows
events has 306514 rows


In [35]:
# Clean the events data
for dic in events.value:
    if list(dic.keys()) == ['offer id']:
        dic['offer_id'] = dic.pop('offer id')

# Concat and transform the events data
temp = pd.DataFrame(events.value.to_list())
events = pd.concat([events,temp],axis=1)
events.rename(columns={'reward':'reward_received', 'amount':'amount_spent'},inplace=True)
events.drop('value',axis=1,inplace=True)
print(f'offers has {len(offers)} rows')
print(f'customers has {len(customers)} rows')
print(f'events has {len(events)} rows')
events.head()

offers has 10 rows
customers has 16994 rows
events has 306514 rows


,customer_id,event,hour,offer_id,amount_spent,reward_received
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN


In [36]:
offers['offer'] = offers.offer_type + '_' + offers.min_spend.astype(str) + '_' + offers.expire_days.astype(str) + '_' + offers.reward.astype(str)
offers.head()

,reward,channels,min_spend,expire_days,offer_type,id,offer
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,bogo_10_7_10
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo_10_5_10
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,informational_0_4_0
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo_5_7_5
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount_20_10_5


In [37]:
customers.age = customers.age.replace({118:np.nan})

# Create new feature for customer data - membership_days
customers.became_member_on = pd.to_datetime(customers.became_member_on, format='%Y%m%d')
most_recent = max(customers.became_member_on)
customers['membership_days'] = customers.became_member_on.apply(lambda x: most_recent - x).dt.days
customers.drop('became_member_on',axis=1,inplace=True)

print(f'offers has {len(offers)} rows')
print(f'customers has {len(customers)} rows')
print(f'events has {len(events)} rows')

offers has 10 rows
customers has 16994 rows
events has 306514 rows


In [38]:
offers.to_csv('cleaned_data/offers.csv',index=False)
customers.to_csv('cleaned_data/customers.csv',index=False)
events.to_csv('cleaned_data/events.csv',index=False)